## Traffic Monitoring System

In [1]:
import cv2
import numpy as np

#cap=vid, frame1=frame, algo=wb

#Web camera
vid=cv2.VideoCapture('video.mp4')                                   #Taking the video as input/Captures video

count_line_pos = 550                                                #Value of yaxis for detection line
min_width_rect=80                                                   #Min width and ht of rect for the object detection
min_ht_rect=80
no=0

wb=cv2.bgsegm.createBackgroundSubtractorMOG()                       #wb = without background
                                                                    #MOG doesnt detect shadows whereas MOG2 detects shadows

def center_handle(x,y,w,h):
    x1=int(w/2)
    y1=int(h/2)
    cx=x+x1
    cy=y+y1
    return cx,cy

detect=[]
offset=3                                                          #Alllowable error between pixel
counter=[0,0]
    
while True:
    ret,frame=vid.read()                                            #reads video frame by frame (frame 1) and returns 
                                                                       #true if frame1 is present in the video
    #framex = frame[y1:y2, x1:x2]
    frame1 = frame[0:600, 0:600]
    frame2 = frame[0:600, 600:1200]
    cv2.line(frame1,(25,count_line_pos),(550,count_line_pos),(255,127,0),3)       #Down going
    cv2.line(frame2,(125,count_line_pos),(550,count_line_pos),(127,255,0),3)       #Down going
    
    f = [frame1, frame2]
    counterShape1=counterShape2=0
    cs = [counterShape1, counterShape2]
    col = [(0,0,255),(255,255,0)]
    
    for i in range (0,2):
        grey=cv2.cvtColor(f[i],cv2.COLOR_BGR2GRAY)
        blur=cv2.GaussianBlur(grey,(3,3),5)

        '''Morphological operations are simple transformations applied to binary or grayscale images. More specifically, 
           we apply morphological operations to shapes and structures inside of images.Morphological operations “probe” 
           an image with a structuring element. This structuring element defines the neighborhood to be examined around 
           each pixel. And based on the given operation and the size of the structuring element we are able to adjust 
           our output image.'''

        #Applying on each frame
        bgsub= wb.apply(blur)                                        #Removes background(black) and shows object(white)
        dilat=cv2.dilate(bgsub,np.ones((5,5)))                         #increases the white region in the image or the size of 
                                                                          #the foreground object increases
        kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))      #to get a circular structuring element
        close=cv2.morphologyEx(dilat,cv2.MORPH_CLOSE,kernel)       
        '''It allows us to pass in whichever morphological operation we want, followed by our kernel/structuring element.
           The first required argument of cv2.morphologyEx is the image we want to apply the morphological operation to. 
           The second argument is the actual type of morphological operation — in this case, it’s an closing operation. 
           (A closing operation is a dilation followed by an erosion. It is used to close holes inside of objects or for 
           connecting components together.) The last required argument is the kernel/structuring element that we are using.'''
        close=cv2.morphologyEx(close,cv2.MORPH_CLOSE,kernel)
        cs[i],h =cv2.findContours(close,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        '''Counter is used for shape analysis and object detection and recognition. It is simply a curve joining all the 
           continuous points (along the boundary), having same color or intensity
           contours, hierarchy = cv2.findcontours(img, mode, method)
           mode – contour retrieval mode
           cv2.RETR_EXTERNAL – retrieves only the extreme outer contours
           cv2.RETR_LIST – retrieves all of the contours
           method – contour approximation method
           cv2.CHAIN_APPROX_NONE – stores all the contour points
           cv2.CHAIN_APPROX_SIMPLE – stores only the corner points
           For more: https://homes.cs.washington.edu/~shapiro/EE596/notes/OpenCV.pdf'''                           
   
        #For drawing rectangles around detected objects
        for (j,c) in enumerate(cs[i]):

            (x,y,w,h)=cv2.boundingRect(c)
            '''Used to draw an approximate rectangle around the binary image. This function is used mainly to 
               highlight the region of interest after obtaining contours from an image.'''

            validate_counter=((w>=min_width_rect) and (h>=min_ht_rect))      #Will avoid detecting vehicles which are 
            if not validate_counter:                                         #far awy from the line
                continue

            cv2.rectangle(f[i],(x,y),(x+w,y+h),col[i],2)
            cv2.putText(f[i],"VEHICLE "+str(no),(x,y-20),cv2.FONT_HERSHEY_TRIPLEX,0.5,(255,244,255),1)
                                                                     #For giving ID to the vehicle
            center=center_handle(x,y,w,h)
            detect.append(center)
            cv2.circle(f[i],center,4,col[i],-1)                 #cv2.circle(image, center_coordinates, radius, color, thickness)
                                                                     #Thickness of -1 fills the circle
                
            for (x,y) in detect:                                     #Count Vehicles
                if y < (count_line_pos + offset) and y > (count_line_pos - offset):
                    counter[i]+=1
                    no+=1
                    cv2.line(f[i],(25,count_line_pos),(650,count_line_pos),(0,127,255),3)
                    detect.remove((x,y))
                    print("Vehicle ",str(no),":",str(counter[i]))   
    
    cv2.putText(frame1,"VEHICLE COUNTER DOWN :"+ str(counter[0]),(50,70),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),4)
    cv2.putText(frame2,"VEHICLE COUNTER UP :"+ str(counter[1]),(50,70),cv2.FONT_HERSHEY_SIMPLEX,1,(127,127,127),4)
            
    cv2.imshow("Footage",frame)                                       #Prints the video frame by frame and title of the 
                                                                      #window is "Footage"
    
    if cv2.waitKey(1)==13:                                            #display a window for given milliseconds(non zero)
                                                                      #and exits on enter(13)
        break

cv2.destroyAllWindows()                                               #If you have multiple windows open at the same time 
                                                                      #and you want to close then you would use this function. 
vid.release()                                                         #Closes video file or capturing device
   

Vehicle  1 : 1
Vehicle  2 : 2
Vehicle  3 : 3
Vehicle  4 : 4
Vehicle  5 : 5
Vehicle  6 : 1
Vehicle  7 : 6
Vehicle  8 : 7
